# One-Photon Calcium Imaging Example

This notebook demonstrates how to use the ndx-microscopy extension for storing one-photon calcium imaging data in NWB format.

## Import Required Libraries

Import necessary packages from NWB, ndx-microscopy, and ndx-ophys-devices extensions, along with other utility libraries.

In [ ]:
from datetime import datetime
from uuid import uuid4
import matplotlib.pyplot as plt
import numpy as np
from pynwb import NWBFile, NWBHDF5IO

from ndx_microscopy import (
    MicroscopeModel,
    Microscope,
    MicroscopyRig,
    PlaneAcquisition,
    MicroscopyChannel,
    PlanarImagingSpace,
    PlanarMicroscopySeries,
    PlanarSegmentation,
    SummaryImage,
    MicroscopyResponseSeries,
    MicroscopyResponseSeriesContainer,
)

from ndx_ophys_devices import (
    ExcitationSourceModel,
    ExcitationSource,
    BandOpticalFilterModel,
    BandOpticalFilter,
    DichroicMirrorModel,
    DichroicMirror,
    PhotodetectorModel,
    Photodetector,
    Indicator
)

## Create NWB File

Initialize a new NWB file with basic session information.

In [ ]:
nwbfile = NWBFile(
    session_description="One-photon calcium imaging session",
    identifier=str(uuid4()),
    session_start_time=datetime.now(),
    lab="Neural Imaging Lab",
    institution="University of Neuroscience",
    experiment_description="GFP imaging in Hippocampus",
)

## Set Up Microscope and Components

Configure the microscope and its various components including:
- Microscope model and instance
- Blue LED excitation source (470nm for GFP)
- Excitation filter (470/40nm bandpass)
- Dichroic mirror (495nm cutoff)
- Emission filter (525/50nm bandpass)
- CMOS camera detector
- GFP indicator

In [ ]:
# Set up microscope model
microscope_model = MicroscopeModel(
    name='1p-model',
    description='One-photon microscope model for calcium imaging',
    model_number='1p-001',
    manufacturer='Custom Build'
)
nwbfile.add_device(microscope_model)

# Set up microscope with technique
microscope = Microscope(
    name='1p-scope',
    description='Custom one-photon microscope for calcium imaging',
    serial_number='1p-serial-001',
    model=microscope_model,
    technique='widefield'
)
nwbfile.add_device(microscope)

In [ ]:
# Configure LED light source model and instance
excitation_source_model = ExcitationSourceModel(
    name="blue_led_model",
    manufacturer="Thorlabs",
    model_number="M470L4",
    description="Blue LED excitation source model for GFP imaging",
    source_type="LED",
    excitation_mode="one-photon",
    wavelength_range_in_nm=[450.0, 490.0]
)
nwbfile.add_device(excitation_source_model)

laser = ExcitationSource(
    name="blue_laser",
    description="Blue LED for one-photon excitation",
    serial_number="LED-SN-123456",
    model=excitation_source_model,
    power_in_W=0.18,  # 180mW typical for this LED
    intensity_in_W_per_m2=1500.0,  # ~1500 W/m² at sample
    exposure_time_in_s=0.001,  # 1ms exposure time
)
nwbfile.add_device(laser)

In [ ]:
# Configure optical components
excitation_filter_model = BandOpticalFilterModel(
    name="excitation_filter_model",
    filter_type="Bandpass",
    manufacturer="Chroma",
    model_number="ET470/40x",
    center_wavelength_in_nm=470.0,
    bandwidth_in_nm=40.0
)
nwbfile.add_device(excitation_filter_model)

excitation_filter = BandOpticalFilter(
    name="excitation_filter",
    description="Excitation filter for one-photon imaging",
    serial_number="EF-SN-123456",
    model=excitation_filter_model
)
nwbfile.add_device(excitation_filter)

dichroic_mirror_model = DichroicMirrorModel(
    name="primary_dichroic_model",
    manufacturer="Chroma",
    model_number="T495lpxr",  # Common dichroic for GFP imaging
    cut_on_wavelength_in_nm=495.0,  # Transmits >495nm
    cut_off_wavelength_in_nm=490.0,  # Reflects <490nm
    transmission_band_in_nm=[495.0, 750.0],  # Transmits emission light
    reflection_band_in_nm=(400.0, 490.0),  # Reflects excitation light
    angle_of_incidence_in_degrees=45.0
)
nwbfile.add_device(dichroic_mirror_model)

dichroic = DichroicMirror(
    name="primary_dichroic",
    description="Dichroic mirror for one-photon imaging",
    serial_number="DM-SN-123456",
    model=dichroic_mirror_model
)
nwbfile.add_device(dichroic)

emission_filter_model = BandOpticalFilterModel(
    name="emission_filter_model",
    filter_type="Bandpass",
    manufacturer="Chroma",
    model_number="ET525/50m",
    center_wavelength_in_nm=525.0,
    bandwidth_in_nm=50.0
)
nwbfile.add_device(emission_filter_model)

emission_filter = BandOpticalFilter(
    name="emission_filter",
    description="Emission filter for GFP",
    serial_number="EF-SN-123456",
    model=emission_filter_model
)
nwbfile.add_device(emission_filter)

photodetector_model = PhotodetectorModel(
    name="camera_model",
    detector_type="CMOS",
    manufacturer="Hamamatsu",
    model_number="ORCA-Flash4.0",
    gain=1.0,
    gain_unit="e-/ADU"
)
nwbfile.add_device(photodetector_model)

detector = Photodetector(
    name="camera",
    description="CMOS camera for fluorescence detection",
    serial_number="CAM-SN-123456",
    model=photodetector_model
)
nwbfile.add_device(detector)

In [ ]:
# Create indicator
indicator = Indicator(
    name="gfp",
    label="GFP",
    description="Green fluorescent protein for one-photon imaging",
    manufacturer="Addgene",
    injection_brain_region="Hippocampus",
    injection_coordinates_in_mm=[-1.8, 2.0, 1.2],
)

## Configure Microscopy Rig

Set up the microscopy rig, connecting all the previously defined components.

In [ ]:
# Create microscopy rig
microscopy_rig = MicroscopyRig(
    name='1p_rig',
    description='One-photon microscopy rig for calcium imaging',
    microscope=microscope,
    excitation_source=laser,
    excitation_filter=excitation_filter,
    dichroic_mirror=dichroic,
    photodetector=detector,
    emission_filter=emission_filter
)

# Create microscopy channel
microscopy_channel = MicroscopyChannel(
    name="GFP_channel",
    description="GFP channel",
    excitation_wavelength_in_nm=470.0,
    emission_wavelength_in_nm=525.0,
    indicator=indicator,
)

## Define Imaging Space and Create Example Data

Set up the imaging space parameters for hippocampal imaging and create sample imaging data.

In [ ]:
# Create example imaging data
frames = 1000
height = 512
width = 512
data = np.random.rand(frames, height, width)  

# Define imaging space
illumination_pattern = PlaneAcquisition(
    name="plane_acquisition",
    description="Widefield fluorescence imaging",
    point_spread_function_in_um="32 um ± 1.6 um",
)

imaging_space = PlanarImagingSpace(
    name="hippo_plane1",
    description="CA1 region of hippocampus",
    pixel_size_in_um=[1.0, 1.0],
    dimensions_in_pixels=[width, height],
    origin_coordinates=[-1.8, 2.0, 1.2],  # Matches the indicator injection coordinates
    location="Hippocampus, CA1 region",
    reference_frame="bregma",
    orientation="RAS",  # Right-Anterior-Superior
    illumination_pattern=illumination_pattern,
)

# Create imaging series
imaging_series = PlanarMicroscopySeries(
    name='imaging_data',
    description='One-photon calcium imaging',
    microscopy_rig=microscopy_rig,
    microscopy_channel=microscopy_channel,
    planar_imaging_space=imaging_space,
    data=data,
    unit="a.u.",
    rate=30.0,
    starting_time=0.0,
)
nwbfile.add_acquisition(imaging_series)

## ROI Segmentation and Analysis

Create and analyze regions of interest (ROIs) in the imaging data:

In [ ]:
# Create ophys processing module
ophys_module = nwbfile.create_processing_module(
    name='ophys',
    description='Optical physiology processing module'
)

# Create summary images
mean_image = SummaryImage(
    name='mean',
    description='Mean intensity projection',
    data=np.mean(data, axis=0)
)

max_image = SummaryImage(
    name='max',
    description='Maximum intensity projection',
    data=np.max(data, axis=0)
)

# Create segmentation
segmentation = PlanarSegmentation(
    name='rois',
    description='Manual ROI segmentation',
    planar_imaging_space=imaging_space,
    summary_images=[mean_image, max_image]
)

In [ ]:
# Add ROIs using image masks
roi_mask = np.zeros((height, width), dtype=bool)
roi_mask[256:266, 256:266] = True  # 10x10 ROI
segmentation.add_roi(image_mask=roi_mask)

# Create ROI responses
roi_region = segmentation.create_roi_table_region(
    description='All ROIs',
    region=list(range(len(segmentation.id)))
)

# Extract responses (example calculation)
num_rois = len(segmentation.id)
responses = np.zeros((frames, num_rois))

for i, roi_mask in enumerate(segmentation.image_mask[:]):
    roi_data = data[:, roi_mask]
    responses[:, i] = np.mean(roi_data, axis=1)

# Create response series
response_series = MicroscopyResponseSeries(
    name='roi_responses',
    description='Fluorescence responses from ROIs',
    data=responses,
    rois=roi_region,
    unit='n.a.',
    rate=30.0,
    starting_time=0.0,
    microscopy_series=imaging_series,
)

# Create container for response series
response_container = MicroscopyResponseSeriesContainer(
    name='responses',
    microscopy_response_series=[response_series]
)

# Add segmentation and responses to ophys module
ophys_module.add(segmentation)
ophys_module.add(response_container)

## Save and Load NWB File

Write the NWB file to disk and demonstrate how to read it back.

In [ ]:
# Save file
with NWBHDF5IO('one_photon_calcium_imaging.nwb', 'w') as io:
    io.write(nwbfile)

# Read file and access data
with NWBHDF5IO("one_photon_calcium_imaging.nwb", "r") as io:
    nwbfile = io.read()

    # Access imaging data
    imaging = nwbfile.acquisition['imaging_data']
    raw_data = imaging.data[:]

    # Access ROI data
    ophys = nwbfile.processing['ophys']
    rois = ophys['rois']
    roi_masks = rois.image_mask[:]

    # Access responses
    responses = ophys['responses']
    roi_data = responses['roi_responses'].data[:]

## Data Visualization

Create plots to visualize the imaging data, ROI masks, and fluorescence responses.

In [ ]:
# Create a figure with 2 rows and 2 columns
fig = plt.figure(figsize=(20, 15))

# First frame
ax1 = plt.subplot(2, 2, 1)
im1 = ax1.imshow(raw_data[0], cmap="gray")
ax1.set_title("First Frame")
plt.colorbar(im1, ax=ax1)

# ROI masks
ax2 = plt.subplot(2, 2, 2)
for i, mask in enumerate(roi_masks):
    im2 = ax2.imshow(mask, alpha=0.5, cmap="viridis")
    ax2.set_title(f"ROI {i+1} Mask")
plt.colorbar(im2, ax=ax2)

# ROI responses over time
ax3 = plt.subplot(2, 1, 2)
ax3.plot(roi_data)
ax3.set_title('ROI Fluorescence Response')
ax3.set_xlabel('Frame Number')
ax3.set_ylabel('Fluorescence (a.u.)')
ax3.grid(True)

plt.tight_layout()
plt.show()